In [17]:
from geopy.distance import geodesic as GD

In [18]:
import numpy as np
import pandas as pd
import csv
from collections import namedtuple
from collections import defaultdict
import math
import time

In [20]:
nyc_taxi_data = pd.read_csv('./nyc_taxi_data.csv')


In [7]:
nyc_taxi_data_filtered = nyc_taxi_data[['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','trip_distance']]


In [21]:
nyc_taxi_round = nyc_taxi_data_filtered.round(4)
weighted_edges = nyc_taxi_round.values.tolist()
for i in weighted_edges:
    nyc1 =(i[0],i[1])
    nyc2 = (i[2],i[3])
    heuristic_val = GD(nyc1,nyc2).km
    i = i.append(heuristic_val)
      

In [9]:
graph = defaultdict(list)
for i in weighted_edges:
    pick_cord=(i[0],i[1])
    graph[pick_cord].append([ i[2],i[3],i[4],i[5]])
graph

In [11]:
i = 0
lat_long_id = {}
for key in graph:
    values = graph[key]
    if key not in lat_long_id:
        i += 1
        lat_long_id[key] = i
    for value in values:
        lat, long = value[0], value[1]
        if (lat,long) not in lat_long_id:
            i += 1
            lat_long_id[lat, long] = i

list_of_nodeid_lat_long = list(lat_long_id.keys())
list_of_nodeid_nodeId = list(lat_long_id.values())
graph

defaultdict(list,
            {(-73.9641, 40.7614): [[-73.978, 40.7831, 2.4, 1.6894855129839041],
              [-73.9919, 40.7696, 1.7, 3.1129786310005434],
              [-73.9731, 40.755, 0.82, 1.0236770122914882],
              [-73.9963, 40.7323, 3.1, 3.7039722533204325],
              [-73.9671, 40.7724, 1.0, 0.47667309932425267]],
             (-73.9615, 40.7604): [[-73.9436, 40.7097, 5.0, 2.537787732633327],
              [-74.0101, 40.7064, 6.4, 5.673508753493558],
              [-73.995, 40.7256, 4.8, 3.8896609100874993],
              [-73.9272, 40.7629, 3.04, 3.828921438174835],
              [-73.9714, 40.7665, 0.8, 1.1208208302241496],
              [-73.9483, 40.7745, 1.66, 1.536152219899045]],
             (-73.9869,
              40.7617): [[-73.8732,
               40.7743,
               10.25,
               12.695749689858845], [-74.1832, 40.688, 20.24, 22.024784284427046]],
             (-73.9658, 40.7581): [[-74.0104, 40.702, 5.46, 5.269113851617224],
           

In [12]:
node_1_2 = []
node_1_2_dist_heu = []
for source in graph:
    destinations = graph[source]
    source_id = lat_long_id[source]
    for destination in destinations:
        # print(destination)
        lat, long = destination[0], destination[1]
        lat, long, dist,heu = destination[0], destination[1],destination[2],destination[3]
        # print(lat,long)
        destination_id = lat_long_id[lat, long]
        node_1_2.append((source_id, destination_id))
        node_1_2_dist_heu.append([source_id, destination_id,dist,heu])

In [13]:
def directedGraph(edges):
    graph_new = defaultdict(dict)
    for u, v, dist,heu in edges:
        graph_new[u][v] = dist,heu
        graph_new[v][u] = dist,heu
    return graph_new

directed_weighted_graph=directedGraph(node_1_2_dist_heu)
directed_weighted_graph

defaultdict(dict,
            {1: {2: (2.4, 1.6894855129839041),
              3: (1.7, 3.1129786310005434),
              4: (0.82, 1.0236770122914882),
              5: (3.1, 3.7039722533204325),
              6: (1.0, 0.47667309932425267)},
             2: {1: (2.4, 1.6894855129839041),
              76693: (1.28, 1.5990345357845408),
              14661: (2.08, 2.4282294077929403),
              24996: (1.63, 1.630724870942278)},
             3: {1: (1.7, 3.1129786310005434)},
             4: {1: (0.82, 1.0236770122914882),
              54894: (1.89, 2.4073265644096646),
              26733: (1.19, 2.020143493962763),
              159: (8.91, 11.232414704732738)},
             5: {1: (3.1, 3.7039722533204325),
              56536: (1.09, 0.4133553837333741),
              59220: (4.18, 1.4593202662391944),
              59221: (2.3, 1.3931543454829078),
              59222: (5.6, 2.973534612065138),
              59223: (2.0, 0.995766518833487),
              59224: (2.37, 2.0762

In [14]:
start = time.time()
def find_heu(cell, goal):
    start_cord = (list(lat_long_id.keys())[list(lat_long_id.values()).index(cell)])
    goal_cord = (list(lat_long_id.keys())[list(lat_long_id.values()).index(goal)])
    heuristic_val = GD(start_cord,goal_cord).km
    return heuristic_val
    
def reconstruct_path(came_from, current):
    final_path = [current]
    while current in came_from:
        current = came_from[current]
        final_path.append(current)
    return final_path


def A_star(graph, start, goal):
    closed_set = []
    open_set = [start]
    came_from ={}
    gscore ={}
    for key in graph:
        gscore[key] = 10000
    gscore[start] = 0
    fscore = {}
    for key in graph:
        fscore[key] = 1000000
    fscore[start] = find_heu(start, goal)
    
    
    while open_set:
        min_val = 10000
        for node in open_set:
            if fscore[node] < min_val:
                min_val = fscore[node]
                min_node = node
        current = min_node
        
        if current == goal:
            return reconstruct_path(came_from, current)
        open_set.remove(current)
        closed_set.append(current)
        
        for neighbor in graph[current]:
            if neighbor in closed_set:
                continue
            if neighbor not in open_set:
                open_set.append(neighbor)
            tentative_gscore = gscore[current] + 1
            
            if tentative_gscore >= gscore[neighbor]:
                continue
            came_from[neighbor] = current  # record the best path untill now
            gscore[neighbor] = tentative_gscore
            fscore[neighbor] = gscore[neighbor] + find_heu(start, goal)
    print("here")
    return False

In [15]:
end = time.time()
shortest_route = A_star(directed_weighted_graph, 1, 10)
time = end - start

In [16]:

dist = 0
for i in range(len(shortest_route) - 1):
    key =shortest_route[i]
    j = i +1
    key2 = shortest_route[j]
    dist = dist + directed_weighted_graph[key][key2][0]
    
print(f'The path by A* is:{shortest_route}')   
print(f'The path distance of A* is :{dist}')    
print(f'The timr taken by A* is:{time}')

path is:[10, 19717, 1936, 1928, 1682, 26733, 4, 1]
path distance is :33.419999999999995
time take is:1.0737495422363281
